In [153]:
!pip install efficientnet_pytorch

In [154]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

In [198]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
model_path = '/content/drive/My Drive/Colab Notebooks/thresh2.pt'
image_path = '/content/drive/My Drive/Colab Notebooks/woo.jpg'

In [214]:
class_names = ['0', '1']

In [ ]:
model = torch.load(model_path)
model.eval()

In [216]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [217]:
model = model.to(device)

In [218]:
transform = transforms.Compose([
    transforms.Resize([224, 224], interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [219]:
# Prediction function
def predict_image_with_weighted_average(image_path, model, transform, class_names):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    input_tensor = input_tensor.to(device)


    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)[0]

    weighted_sum = 0
    for i, prob in enumerate(probabilities):
        class_probability = prob.item()
        weighted_value = i * class_probability
        weighted_sum += weighted_value
        print(f"Class {class_names[i]} (index {i}) Probability: {class_probability*100:.2f}%")

    print(f"Weighted sum of predictions: {weighted_sum}")

    return weighted_sum

In [222]:
weighted_sum = predict_image_with_weighted_average(image_path, model, transform, class_names)

Class 0 (index 0) Probability: 100.00%
Class 1 (index 1) Probability: 0.00%
Weighted sum of predictions: 1.2490851686308702e-09
